In [ ]:
import glob
import math
import uproot
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil
import uproot_methods

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["svg"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

plt.style.use(hep.style.ROOT)  # For now ROOT defaults to CMS
# hep.set_style({"font.size":8})
# mpl.rc('font', size=8)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV';en="KE=3eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV';en="KE=4eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV';en="KE=6.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV';en="KE=8.2eV";
path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

#if True it computes the 72 MFPAD
MFPAD=False

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
"""
##############################################NOTE####################################################
in elements tag:
cosphi_th=([[ 1,  1],   cosphi_th_cos=([[ 6,  1],       cosphi_phi=([[ 6,  1],      
            [ 1,  2],                   [ 6,  2],                    [ 5,  1],                   
            [ 1,  3],                   [ 6,  3],                    [ 4,  1],
            [ 1,  4],                   [ 6,  4],                    [ 3,  1],
            [ 1,  5],                   [ 6,  5],                    [ 2,  1],                   
            [ 1,  6],                   [ 6,  6],                    [ 1,  1],

in angles tag:
cosphi_adj_th=([[ 1,  -180],   cosphi_adj_cos=([[ -1,  -180],       cosphi_adj_phi=([[ -1,   -180],      
                [ 1,  -147],                    [ -1,  -147],                        [ -0.8, -180],                   
                [ 1,  -114],                    [ -1,  -114],                        [ -0.3, -180],
                [ 1,  -81],                     [ -1,  -81],                         [ 0.3,  -180],
                [ 1,  -49],                     [ -1,  -49],                         [ 0.8,  -180],                   
                [ 1,  -16],                     [ -1,  -16],                         [ 1,    -180],

cosphi_th has to be used JUST with counts_stemp!!

b1 maps cordinates. YOU SHOULD USE THIS AS ROTATION INPUT

list(zip(xgo_phi,ygo_phi))             list(zip(xgo_cos,ygo_cos))
[(-180.0, 1.0),                        [(-180.0, 1.0),
 (-180.0, 0.809),                       (-147.273, 1.0),
 (-180.0, 0.309),                       (-114.545, 1.0),
 (-180.0, -0.309),                      (-81.818, 1.0),
 (-180.0, -0.809),                      (-49.091, 1.0),
 (-180.0, -1.0),                        (-16.364, 1.0),
##############################################NOTE####################################################
######################################################################################################
                                    PHI - CTHETEA ELECTRON 
######################################################################################################
(alpha and beta in Philipp convention)
REMEBER: in Philipp simulations cos(theta)=[1,-1] becase theta=[0,np.pi]
SORTED = cos(theta)=[-1,1] IS NEVER USED for electrons coordinates
#NOTE: the following phi and ctheta are not unique: len=20000

"""
#SORTED FULL **cos(theta) = [-1,1]**, phi = [-179.1, 179.1]
# len = 200000
phi_full_sorted=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["phi"] 
ctheta_full_sorted=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["cos(theta)"]

#SORTED unique **cos(theta) = [-1,1]**, phi = [-179.1, 179.1]  
# len(phiMs) = 200, len(cosMs) = 100
phiMs=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["phi"].unique()
cosMs=frame_srt.get_group("1_1").sort_values(by=["cos(theta)","phi"])["cos(theta)"].unique()

#UNSORTED FULL phi = [-179.1, 179.1]
# len = 200000
phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
phi_rad = phi_full * np.pi/180.

#UNSORTED FULL cos(theta) = [1,-1]
# len = 200000
ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)
theta_rad = np.arccos(ctheta_full)
theta_full = theta_rad*180./np.pi

#UNSORTED unique cos(theta) = [1,-1], phi = [-179.1, 179.1]
# len(phiMs) = 200, len(cosMs) = 100
#NOTE      np.any(phiM == phiMs) -> TRUE
#          np.any(cosM == np.flip(cosMs)) -> TRUE
phiM = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi = [-179.1, 179.1]
cosM = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta) = [1,-1]

#NOTE: for np.MESHGRID:
#for the 2D case with inputs of length M and N, the outputs are of shape (N, M) for ‘xy’ indexing and (M, N) for ‘ij’ indexing. Cartesia "xy" by default
#REMEBER: np.any(phiM == phiMs) -> TRUE

#This SHOULDN´T be use to inrepolate
#because **cos(theta) = [-1,1]**
phiMMs, cosMMs = np.meshgrid(phiMs, cosMs)
phiMMs_rad, thetaMMs_rad = np.meshgrid(phiMs*np.pi/180., np.arccos(cosMs))

#This is the right one to use to interpolate
# because cos(theta)=[1,-1]
phiMM, cosMM = np.meshgrid(phiM, cosM)
phiMM_rad, thetaMM_rad = np.meshgrid(phiM*np.pi/180., np.arccos(cosM))

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print("pihMs ", phiMM.shape, "cosMs ", cosMM.shape)
print(type(phiM), type(phiMM))

In [ ]:
"""
##########################b1 map x,y coordiantes###############################
PHI - CTHETA photon with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, reduced=False, source=True)

# list(zip(xgo_phi,ygo_phi))
print("(xgo_cos (phi), ygo_cos (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("\n cosphi_adj_cos  \n", cosphi_adj_cos[:6])
print("\n cosphi_adj_phi \n", cosphi_adj_phi[:6])

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
counts_temp=[] #standard cos sorted S enantiomer
MFPAD_R_phi=[]
MFPAD_R_cos=[]

#NOTE the values of each MFPAD is espressend with phi[-180,180], cos(theta)[1,-1] for ELECTRONS
#counts_temp: in photon_coordiantes cos(theta)=[1,-1], phi=[-180,180], ordered by cos(theta)
for group in frame_srt:
    # elsorted=group[1].sort_values(by=["cos(theta)","phi"]) #just a try to remap cos(theta)
    # counts_temp.append(elsorted["value"].values.reshape(200,100))
    counts_temp.append(group[1]["value"].values.reshape(200,100))
counts_temp=np.array(counts_temp)

#MFPAD_S_phi: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] ordered by phi
MFPAD_S_phi=fugi.sorting_array(counts_temp, theory=True, items=cosphi_th, a=2)
#MFPAD_S_cos: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] ordered by cos
MFPAD_S_cos=fugi.sorting_array(counts_temp, theory=True, items=cosphi_th, a=1)

"""
Whith this sorting:
MFPAD_S_phi == MFPAD_S_cos gives True for the first and the last element
MFPAD_S_phi or MFPAD_S_phi == counts_temp are False, because the cos(theta) PHOTON is changed to [-1,1] from the original [1,-1]
"""

#flipping for R enantiomer and sorting: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_S_phi:
    MFPAD_R_phi.append(np.flip(el, axis=1))
    # MFPAD_R_phi.append(np.flip(MFPAD_R_phi, axis=1))

for el in MFPAD_S_cos:
    MFPAD_R_cos.append(np.flip(el,axis=1))
    # MFPAD_R_cos.append(np.flip(MFPAD_R_cos, axis=1))


In [ ]:
MFPAD_R_phi_LF=[]
MFPAD_R_cos_LF=[]

#rotation of the MFPAD according to the photon rotation and triangulating the result on a ADJUSTED meshgrid phiM, cosMM
#IS THE GRID ACCORDING TO THE COS(THETA) PHI OF THE ORIGINAL? I.E. y= 1,-1, x= -180,180
MFPAD_S_phi_LF,cos_S_phi_LF,phi_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
MFPAD_S_cos_LF,cos_S_cos_LF,phi_S_cos_LF=fugi.rot3d_MFPAD(MFPAD_S_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")

# MFPAD_R_phi_LF,cos_R_phi_LF,phi_R_phi_LF=fugi.rot3d_MFPAD(MFPAD_R_phi,theta_rad,phi_rad,cosphi_adj_phi,phiMM,cosMM,method="linear")
# MFPAD_R_cos_LF,cos_R_cos_LF,phi_R_cos_LF=fugi.rot3d_MFPAD(MFPAD_R_cos,theta_rad,phi_rad,cosphi_adj_cos,phiMM,cosMM,method="linear")

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_S_phi_LF:
    MFPAD_R_phi_LF.append(np.flip(el, axis=1))
    # MFPAD_R_phi_LF.append(np.flip(MFPAD_R_phi_LFk, axis=1))

for el in MFPAD_S_cos_LF:
    MFPAD_R_cos_LF.append(np.flip(el, axis=1))
    # MFPAD_R_cos_LF.append(np.flip(MFPAD_R_cos_LF, axis=1))


In [ ]:
"""
Creation of DataFrame to find the right element according to rotation
isuefull fot the 3d representation
NOTE allnames are UNSORTED: provide cosphi_df with cos(thea) = [1,-1] sorted by cos(theta)
NOTE all the coordinates are FULL!! this is iportant for rotation

according to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
according to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

allnames  cos(theta)   phi
1_1       1            -180 
1_2       1            -147 
"""
#NOTE cosphi_adj_cos comes from create_gocoords, therefore can be reduced to meet the experiments, or not
dfind=pd.DataFrame(cosphi_adj_cos, columns=["ctheta","phi"])
dfind["item"]=allnames

#Sorting the label of the element according to the numerical value of ctheta and phi
#NOTE: BOTH will have cos(theta) = [-1,1], regardless from the sorting
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values

In [ ]:
# find the photon rotation vakues of a specific set of rotation
# where b1 is ma or min!
# dfind.sort_values(by=["phi", "ctheta"])
dfind.sort_values(by=["ctheta","phi"])
dfind.loc[dfind["item"] == "4_7"]
dfind.loc[dfind["phi"] == 81.818]

In [ ]:
#selecting the rght file according to the needs ["filename"] == "x_x"]
#NOTE select S or R depending on the needs. 
nitem=32
# counts = MFPAD_S_cos_LF[nitem].reshape(-1)
# counts = MFPAD_R_cos_LF[nitem].reshape(-1)
counts = MFPAD_S_cos[nitem].reshape(-1)

In [ ]:
fig,ax=plt.subplots()
titel_fig="MFPAD el "+str(nitem)
ax.contourf(phiM,cosM,counts.reshape(200,100).T, cmap="viridis")
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
ax.set_title(titel_fig)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig.show()
fig.savefig("OUTPUTS/Theory/MFPAD_el"+str(nitem)+"_"+en+"_theory.png")

In [ ]:
# convertion to spherical coordinates 1D vectors of shape (20000,1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(200,100)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(200,100)
z = counts * np.cos(theta_rad)
Z = z.reshape(200,100)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

#ax.plot(counts_fixed[0:200]-counts[0:200],c="b",label='diff_counts[0:200]')
ax.plot(counts_fixed-counts,c="b",label='diff_counts')
ax.plot(a1-b1,c="g",label='diff_+phi_-phi')
ax.plot(a2-b2,c="y",label='diff_+ctheta_-ctheta')
#ax.plot(avgcos[0:200]-counts[0:200],c="r",label='diff_+avgcos_-counts[0:200]')
#ax.plot(avgphi[0:200]-counts[0:200],c="m",label='diff_+avgphi_-counts[0:200]')
#ax.set_yscale('log')
plt.legend(loc="best")
print(np.min(phi_fixed),np.max(phi_fixed),np.min(ctheta_fixed),np.max(ctheta_fixed))

In [ ]:
fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z,
                      opacity=0.5,
                      color="rgba(244,22,100,06)"
                 
 )])
fig.update_layout(title="TFMeOx MFPADs theory triangulated",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=ctheta_full, y=phi_full, z=counts,
                    mode='markers',
                    # mode='lines',
                    marker=dict(
                    size=1.5,
                    colorscale='Viridis',   # choose a colorscale
                    opacity=0.85,
                    showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs spherical coordinates",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()
fig.write_html("../PYTHON_graphs/OUTPUTS/3D_spherical_"+en+"_theory.html")

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=d,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs theory scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()


the Z distribution has the ending and the starting data that aren't matching. This is regardless the reshaping

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, colorscale='Viridis', connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory sur NO FIXf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

fig.write_html("../PYTHON_graphs/OUTPUTS/3D_cartesian_"+en+"_theory.html")

It is possible to force the equality and therefore bridging the gap

In [ ]:
#fixing the stripe
Y[-1,0:100]=Y[0,0:100]
X[-1,0:100]=X[0,0:100]
Z[-1,0:100]=Z[0,0:100]

#to share these variables to other notebooks
%store X
%store Y
%store Z
%store d_matrix

In [ ]:
#fixed stripe
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                #   width=500, height=500,
                #   margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()